# Introduction

WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. WeRateDogs has over 4 million followers and has received international media coverage.

Your goal: wrangle WeRateDogs Twitter data to create interesting and trustworthy analyses and visualizations. The Twitter archive is great, but it only contains very basic tweet information. Additional gathering, then assessing and cleaning is required for "Wow!"-worthy analyses and visualizations.

## Gather

Here i will gather the required data from the different sources. My different sources are:
APi, .csv, .tsv.  
- the csv is loaded through read_csv method
- the tsv is downloaded programmatically
- the api data is queried and saved

In [6]:
import pandas as pd
import numpy as np
# import rtsimple as rt
import requests
import tweepy
import tweepy.api as api
import json
from timeit import default_timer as timer
%run 'authorization.py' #importing macros from python file

In [2]:
#here load data form the csv and tsv

twitter_archive_df = pd.read_csv('twitter-archive-enhanced.csv') #
# image_predictions_1 = pd.read_csv('image-predictions-1.tsv', sep='\t')

In [3]:
#downloading file programmatically

url='https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response= requests.get(url)
with open('image-predictions.tsv','wb') as file:
        file.write(response.content)

In [4]:
image_predictions_df = pd.read_csv('image-predictions.tsv', sep='\t')  #what breed the dog is

In [7]:
#  twitter API authentication.
#have to use glob file here.

#trial wtih one tweet id.
consumer_key = API_KEY 
consumer_secret = API_SECRET 
access_token = ACCESS_TOKEN
access_secret = ACCESS_TOKEN_SECRET 

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify = True)

In [ ]:
#data extraction from twitter api using tweet id from twitter_archive
fails_dict={}
            
lists =['666029285002620928','891815181378084864','892177421306343426']
start = timer()    
with open('tweet_json.txt', 'w') as outfile:
#     for tweet_id in twitter_archive.tweet_id[:100]:
    for tweet_id in twitter_archive.tweet_id:
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
#             print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict) #how to handle fails

In [ ]:
print(fails_dict) 

Some tweet querins have failed. There are only two errors which are recurring:
- TweepError([{'code': 144, 'message': 'No status found with that ID.'}]) :Corresponds with HTTP 404. The requested Tweet ID is not found (if it existed, it was probably deleted)
- TweepError([{'code': 179, 'message': 'Sorry, you are not authorized to see this status.'}]) :Corresponds with HTTP 403. Thrown when a Tweet cannot be viewed by the authenticating user, usually due to the Tweet’s author having protected their Tweets.

In [ ]:
# fails_dict
#how to handle fails
tweet_ids = twitter_archive.tweet_id.values
len(tweet_ids)

In [8]:
#df to store additional info
count= 0
twitter_api = {}
twitter_api['tweet_id']=[]
twitter_api['retweet_count']=[]
twitter_api['favorite_count']=[]
with open('tweet_json.txt', 'r') as file:
#     print(file.readline())
#     j = json.loads(file.readline())
#     print(j['id_str'], j['retweet_count'],j['favorite_count'])
    for line in file.readlines():
        j = json.loads(line)
        count+=1
#         print(str(count)+': ' +j['id_str'],j['retweet_count'],j['favorite_count'])
        twitter_api['tweet_id'].append(j['id_str'])
        twitter_api['retweet_count'].append(j['retweet_count'])
        twitter_api['favorite_count'].append(j['favorite_count'])


In [9]:
twitter_api_df = pd.DataFrame(twitter_api)

## Assess

In [10]:
twitter_archive_df

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None


In [11]:
twitter_archive_df.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1542,689599056876867584,NaN,NaN,2016-01-20 00:03:21 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we see 33 dogs posing for a picture. All ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/689599056...,11,10,None,None,None,None,None
1370,702321140488925184,NaN,NaN,2016-02-24 02:36:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Please enjoy this picture as much as I did. 12...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/702321140...,12,10,None,None,None,None,None
1957,673583129559498752,NaN,NaN,2015-12-06 19:21:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Sandy. She loves her spot by the tree....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/673583129...,11,10,Sandy,None,None,None,None
1454,695446424020918272,NaN,NaN,2016-02-05 03:18:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",We normally don't rate unicorns but this one h...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/695446424...,12,10,None,None,None,None,None
1376,701805642395348998,NaN,NaN,2016-02-22 16:27:58 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",Please pray for this pupper. Nothing wrong wit...,NaN,NaN,NaN,https://vine.co/v/ivV6Y37mH5Z,11,10,None,None,None,pupper,None


In [12]:
twitter_archive_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [13]:
twitter_archive_df.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [14]:
twitter_archive_df.name.value_counts()

None       745
a           55
Charlie     12
Lucy        11
Oliver      11
          ... 
Daniel       1
Clarq        1
Kloey        1
Iggy         1
Dug          1
Name: name, Length: 957, dtype: int64

In [15]:
twitter_archive_df.tweet_id.duplicated().any()

False

In [16]:
image_predictions_df

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True


In [17]:
image_predictions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [18]:
image_predictions_df.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [19]:
image_predictions_df.tweet_id.duplicated().any()

False

In [20]:
twitter_api_df

,tweet_id,retweet_count,favorite_count
0,892420643555336193,7532,35553
1,892177421306343426,5587,30751
2,891815181378084864,3702,23125
3,891689557279858688,7714,38855
4,891327558926688256,8316,37126
...,...,...,...
2326,666049248165822465,40,95
2327,666044226329800704,126,265
2328,666033412701032449,39,110
2329,666029285002620928,41,119


In [21]:
twitter_api_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet_id        2331 non-null   object
 1   retweet_count   2331 non-null   int64 
 2   favorite_count  2331 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 54.8+ KB


In [22]:
twitter_api_df.describe()

,retweet_count,favorite_count
count,2331.000000,2331.000000
mean,2648.198198,7429.762763
std,4478.608707,11538.132656
min,1.000000,0.000000
25%,537.000000,1292.000000
50%,1239.000000,3229.000000
75%,3074.500000,9096.000000
max,76087.000000,153457.000000


In [23]:
twitter_api_df.tweet_id.duplicated().any()

False

#### Quality Issues:
##### `twitter_archive_df` table
- Remove tweets that are replies or retweets
- Missing information (expanded_urls)
- Entry as `None` instead of NaN (dog stage variables, name)
- Erroneous datatypes (tweet_id)
- Incorrect value of rating denominator (must be always = 10, not 0 or 170) **There are multiple dogs in pic, divide both numerator and denominator by no.of dogs in pic.**
- Invalid entries in name column ('a','the',etc.).

##### `image_predictions_df` table
- Erroneous datatypes (tweet_id)
- Algorithm outcome is present.
- Inconsistency in font (p1,p2,p3)

##### `twitter_api_df` table


#### Tidiness Issues:
- One variable in four columns in `twitter_archive_df` table (doggo, floofer, puppo and pupper)
- `twitter_api_df` table should be a part of `twitter_archive_df` table
- Given tweet_id in `twitter_archive_df` table duplicated in `twitter_api_df` and `image_predictions_df` tables


- date and time are mixed in the timestamp column.

## Clean

In [339]:
t_archive_clean = twitter_archive_df.copy()
t_api_clean = twitter_api_df.copy()
image_predictions_clean = image_predictions_df.copy()

### Missing Data

#### `twitter_archive_df`: Some expanded urls are missing. I don't think I will do any analysis with the expanded urls, so we can ignore it for now.

### Quality

#### Remove retweets and replies from the `twitter_archive_df` table

##### Define

Wherever the retweet id or reply param is populated remove that observation row.

##### Code

In [340]:
#get indexes with non null value of in_reply_to_status_id or/and retweeted_status_id
reply_tweets =t_archive_clean[t_archive_clean.in_reply_to_status_id.notna()].index.tolist()
retweeted_tweets =t_archive_clean[t_archive_clean.retweeted_status_id.notna()].index.tolist()

In [341]:
#drop non-original tweets
t_archive_clean.drop(reply_tweets, axis =0, inplace =True)
t_archive_clean.drop(retweeted_tweets, axis =0, inplace =True)

##### Test

In [342]:
t_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2097 non-null   int64  
 1   in_reply_to_status_id       0 non-null      float64
 2   in_reply_to_user_id         0 non-null      float64
 3   timestamp                   2097 non-null   object 
 4   source                      2097 non-null   object 
 5   text                        2097 non-null   object 
 6   retweeted_status_id         0 non-null      float64
 7   retweeted_status_user_id    0 non-null      float64
 8   retweeted_status_timestamp  0 non-null      object 
 9   expanded_urls               2094 non-null   object 
 10  rating_numerator            2097 non-null   int64  
 11  rating_denominator          2097 non-null   int64  
 12  name                        2097 non-null   object 
 13  doggo                       2097 

###### *The columns related to replies and retweets are not needed anymore as they only contain null values now.*

In [343]:
#drop the columns related to replies and retweets
columns=['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id',
         'retweeted_status_timestamp']
t_archive_clean.drop(columns, axis =1, inplace =True)

In [344]:
t_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_id            2097 non-null   int64 
 1   timestamp           2097 non-null   object
 2   source              2097 non-null   object
 3   text                2097 non-null   object
 4   expanded_urls       2094 non-null   object
 5   rating_numerator    2097 non-null   int64 
 6   rating_denominator  2097 non-null   int64 
 7   name                2097 non-null   object
 8   doggo               2097 non-null   object
 9   floofer             2097 non-null   object
 10  pupper              2097 non-null   object
 11  puppo               2097 non-null   object
dtypes: int64(3), object(9)
memory usage: 213.0+ KB


#### Entry 'None' instead of NaN in the `twitter_archive_df` table

##### Define

Set the None values to np.nan in all the 4 dog stage columns.

##### Code

In [345]:
# twitter_archive_df_clean = 
t_archive_clean.replace('None',np.nan, inplace =True)

##### Test

In [346]:
t_archive_clean.eq('None').any()

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


tweet_id              False
timestamp             False
source                False
text                  False
expanded_urls         False
rating_numerator      False
rating_denominator    False
name                  False
doggo                 False
floofer               False
pupper                False
puppo                 False
dtype: bool

#### Erroneous datatype (tweet_id) in the `twitter_archive_df` table

##### Define

Convert the datatype of tweet_id column to string or object type instead of int, since no calculations are to 
be done on this numerical data.

##### Code

In [347]:
# twitter_archive_df_clean = 
t_archive_clean.tweet_id = t_archive_clean.tweet_id.astype(str)

##### Test

In [348]:
t_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_id            2097 non-null   object
 1   timestamp           2097 non-null   object
 2   source              2097 non-null   object
 3   text                2097 non-null   object
 4   expanded_urls       2094 non-null   object
 5   rating_numerator    2097 non-null   int64 
 6   rating_denominator  2097 non-null   int64 
 7   name                1494 non-null   object
 8   doggo               83 non-null     object
 9   floofer             10 non-null     object
 10  pupper              230 non-null    object
 11  puppo               24 non-null     object
dtypes: int64(2), object(10)
memory usage: 213.0+ KB


#### Incorrect value of rating denominator in the `twitter_archive_df` table

##### Define

The value of denominator should ideally be 10 but it is not in all observstions because some pictures contain more than 1 dog. To fix this find out the number of dogs in a pic and then divide numerator and denominator by that number.
We can temporarily combine image number, tweet id and rating denominator, numerator columns and do the calculation.

##### Code

In [349]:
#further inspection
pd.options.display.max_colwidth = 200
t_archive_clean.query('rating_denominator != 10')[['tweet_id','rating_denominator','rating_numerator','text']] 

# fix the denominators where you can and leave the rest out of analysis(remove)
# to fix manually read the text and input correct values using iloc with tweet_id
# b =a.text.str.extract('((?:\d+\.)?\d+)\/(\d+)', expand=True)
# b

,tweet_id,rating_denominator,rating_numerator,text
433,820690176645140481,70,84,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd
516,810984652412424192,7,24,Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx
902,758467244762497024,150,165,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE
1068,740373189193256964,11,9,"After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ"
1120,731156023742988288,170,204,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv
1165,722974582966214656,20,4,Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a
1202,716439118184652801,50,50,This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq
1228,713900603437621249,90,99,Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1
1254,710658690886586372,80,80,Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12
1274,709198395643068416,50,45,"From left to right:\nCletus, Jerome, Alejandro, Burp, &amp; Titson\nNone know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK"


The text contains the correct rating as there are some images in the tweet where there is more than one dog , so the ratings are inflated and can be corrected by dividing the numerator and denominator by the number of dogs.

In [351]:
t_archive_clean.loc[t_archive_clean.tweet_id == '820690176645140481', 'rating_numerator'] = 84/7

t_archive_clean.loc[t_archive_clean.tweet_id == '810984652412424192', 'rating_numerator'] = np.nan

t_archive_clean.loc[t_archive_clean.tweet_id == '758467244762497024', 'rating_numerator'] = 165/15

t_archive_clean.loc[t_archive_clean.tweet_id == '740373189193256964', 'rating_numerator'] = 14

t_archive_clean.loc[t_archive_clean.tweet_id == '731156023742988288', 'rating_numerator'] = 204/17

t_archive_clean.loc[t_archive_clean.tweet_id == '722974582966214656', 'rating_numerator'] = 13

t_archive_clean.loc[t_archive_clean.tweet_id == '716439118184652801', 'rating_numerator'] = 11

t_archive_clean.loc[t_archive_clean.tweet_id == '713900603437621249', 'rating_numerator'] = 99/9

t_archive_clean.loc[t_archive_clean.tweet_id == '710658690886586372', 'rating_numerator'] = 80/8

t_archive_clean.loc[t_archive_clean.tweet_id == '709198395643068416', 'rating_numerator'] = 45/5

t_archive_clean.loc[t_archive_clean.tweet_id == '704054845121142784', 'rating_numerator'] = 60/5

t_archive_clean.loc[t_archive_clean.tweet_id == '697463031882764288', 'rating_numerator'] = 44/4

t_archive_clean.loc[t_archive_clean.tweet_id == '684222868335505415', 'rating_numerator'] = 121/11

t_archive_clean.loc[t_archive_clean.tweet_id == '682962037429899265', 'rating_numerator'] = 10

t_archive_clean.loc[t_archive_clean.tweet_id == '677716515794329600', 'rating_numerator'] = 144/12

t_archive_clean.loc[t_archive_clean.tweet_id == '675853064436391936', 'rating_numerator'] = 88/8

t_archive_clean.loc[t_archive_clean.tweet_id == '666287406224695296', 'rating_numerator'] = 9


In [355]:
# t_archive_clean.rating_denominator.astype(int)
# t_archive_clean.info()

In [ ]:
#setting all denominators as 10
t_archive_clean.rating_denominator = 10

##### Test

In [356]:
# t_archive_clean.query('rating_denominator != 10')[['tweet_id','rating_denominator','rating_numerator','text']] 

In [359]:
t_archive_clean.query('rating_denominator != 10').tweet_id.any()

False

#### Invalid entries in name column ('a','the',etc.). in the `twitter_archive_df` table

##### Define

Most of the names can be extracted from the text given. It requires some complex regex pattern and not all text columns contain dog name.

First find the rows with invalid names, then from the text extract their names.

##### Code

In [602]:
mask = t_archive_clean.name.str.islower().fillna(False)
column_name = 'name'
t_archive_clean.loc[mask, column_name] = np.nan

In [603]:
t_archive_clean[t_archive_clean.name.isna()].sample(5)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1304,707411934438625280,2016-03-09 03:45:22 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","""I shall trip the big pupper with leash. Big pupper will never see it coming. I am a genius."" Both 11/10 https://t.co/uQsCJ8pf51",https://twitter.com/dog_rates/status/707411934438625280/photo/1,11.0,10,NaN,NaN,NaN,pupper,NaN
172,858843525470990336,2017-05-01 00:40:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",I have stumbled puppon a doggo painting party. They're looking to be the next Pupcasso or Puppollock. All 13/10 would put it on the fridge https://t.co/cUeDMlHJbq,https://twitter.com/dog_rates/status/858843525470990336/photo/1,13.0,10,NaN,doggo,NaN,NaN,NaN
852,765395769549590528,2016-08-16 03:52:26 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is my dog. Her name is Zoey. She knows I've been rating other dogs. She's not happy. 13/10 no bias at all https://t.co/ep1NkYoiwB,https://twitter.com/dog_rates/status/765395769549590528/photo/1,13.0,10,NaN,NaN,NaN,NaN,NaN
1560,688519176466644993,2016-01-17 00:32:18 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This pupper is sprouting a flower out of her head. 12/10 revolutionary af https://t.co/glmvQBRjv4,https://twitter.com/dog_rates/status/688519176466644993/photo/1,12.0,10,NaN,NaN,NaN,pupper,NaN
1994,672604026190569472,2015-12-04 02:31:10 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a baby Rand Paul. Curls for days. 11/10 would cuddle the hell out of https://t.co/xHXNaPAYRe,https://twitter.com/dog_rates/status/672604026190569472/photo/1,11.0,10,NaN,NaN,NaN,NaN,NaN


In [591]:
a=t_archive_clean[t_archive_clean.name.isna()]
a.count()

tweet_id              707
timestamp             707
source                707
text                  707
expanded_urls         704
rating_numerator      707
rating_denominator    707
name                    0
doggo                  41
floofer                 5
pupper                105
puppo                   8
dtype: int64

In [601]:
import re
dog_name=[]
name=''

a=t_archive_clean[t_archive_clean.name.isna()]

for ind in t_archive_clean[t_archive_clean.name.isna()].index:
    text = t_archive_clean['text'][ind]
    n= re.search('[Tt]his is\s([A-Z]\w+)',text) or re.search('[Hh]is name is\s([A-Z]\w+)',text) or re.search('[Tt]his is .+ named ([A-Z]\w+)',text) 
#     or re.search('[Tt]hat is ([A-Z]\w+)', text) # only one dog name extracted
#     print(t_archive_clean['name'][ind], t_archive_clean['text'][ind])
#     print(df['name'][ind], df['Stream'][ind]) 
    if n:
        print(n.group(1))
        t_archive_clean['name'][ind] = n.group(1)    

In [568]:
# the twwet ids listed in this cell can be cleaned manually
t =t_archive_clean[t_archive_clean.tweet_id == '668142349051129856'].text
t.str.extract('[Tt]his is .+ named ([A-Z]\w+)',expand=True)
# (r'^[Tt]his name is\s(.+)',t)
t
# 675706639471788032
# 826204788643753985
# 669037058363662336
# 668142349051129856
# 758041019896193024

# 670303360680108032
# 675706639471788032

2237    This lil pup is Oliver. Hops around. Has wings but doesn't fly (lame). Annoying chirp. Won't catch tennis balls 2/10 https://t.co/DnhUw0aBM2
Name: text, dtype: object

In [570]:
import re
dog_name=[]
name=''
a=t_archive_clean[t_archive_clean.name.isna()].text
# pd.options.display.max_colwidth = 200

for text in a:
#     print (a)
    name = re.search('[Tt]his is\s([A-Z]\w+)',text) or re.search('[Hh]is name is\s([A-Z]\w+)',text) or re.search('[Tt]his is .+ named ([A-Z]\w+)',text) or re.search('[Tt]hat is ([A-Z]\w+)', text)
#     or re.search('names are ([A-Z]\w+) ,?([A-Z]\w+)?and ([A-Z]\w+)',text)  ## only on tweet name extracted.
#     name=re.search('[Mm]eet ([A-Z]\w+)',text) or re.search('[Ss]ay hello to ([A-Z]\w+) and (.+)', text)
    if name:
        dog_name.append(name.group(1))
dog_name
# a

['Howard',
 'Blue',
 'Quizno',
 'Bretagne',
 'Wylie',
 'Kip',
 'Jacob',
 'Rufus',
 'Spork',
 'Cherokee',
 'Hemry',
 'Alphred',
 'Alfredo',
 'Leroi',
 'Berta',
 'Chuk',
 'Alfonso',
 'Cheryl',
 'Jessiga',
 'Klint',
 'Big',
 'Kohl',
 'Daryl',
 'Pepe',
 'Octaviath',
 'Johm']

##### Test

In [599]:
a=t_archive_clean[t_archive_clean.name.isna()]
a.count()

tweet_id              681
timestamp             681
source                681
text                  681
expanded_urls         678
rating_numerator      681
rating_denominator    681
name                    0
doggo                  40
floofer                 5
pupper                105
puppo                   8
dtype: int64

#### Erroneous datatypes (tweet_id) in the `image_predictions_df` table

##### Define

Convert the datatype of tweet_id column to string or object type instead of int, since no calculations are to be done on this numerical data.

##### Code

In [277]:
image_predictions_clean.tweet_id = image_predictions_clean.tweet_id.astype(str)

##### Test

In [546]:
image_predictions_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


### Tidiness

#### One variable in four columns in `twitter_archive_df` table (doggo, floofer, puppo and pupper)

##### Define

Concatenate all 4 dog stage columns into a single column 'dog_stage'. This way we can preserve multiple dog stage values in a given row. Then remove the 4 dog stage variable columns.

##### Code

In [ ]:
#concatenating contents of the columns: doggo, puppo, pupper, floofer, while ignoring NaN
t_archive_clean['dog_stage']=t_archive_clean[['doggo', 'pupper', 'puppo','floofer']].apply(lambda x:','.join(x.dropna().values.tolist()), axis=1)

In [ ]:
#replace null string in dog_stage column with NaN
t_archive_clean.dog_stage.replace('',np.nan, inplace =True)

In [ ]:
#delete the columns doggo, puppo, pupper, floofer
t_archive_clean.drop(['doggo', 'puppo', 'pupper', 'floofer'],axis=1,inplace=True)

##### Test

In [ ]:
t_archive_clean.eq('').any()

In [ ]:
t_archive_clean.dog_stage.value_counts()

In [ ]:
t_archive_clean.info()

#### `twitter_api_df` table should be a part of `twitter_archive_df` table

##### Define

Merge the *twitter_api_df* column to the `twitter_archive_df` table, joining on *tweet_id*.

##### Code

In [ ]:
t_archive_clean = pd.merge(t_archive_clean, t_api_clean,
                            on=['tweet_id'], how='left')

##### Test

In [ ]:
t_archive_clean

#### Given tweet_id in `twitter_archive_df` table duplicated in `twitter_api_df` and `image_predictions_df` tables

##### Define

twitter_api_df table is no longer needed so ignore that part. Isolate the tweet_id in the `patients` table, then convert these names to lower case to join with `treatments`. Then drop the given name and surname columns in the treatments table (so these being lowercase isn't an issue anymore).

##### Code

In [ ]:
id_names = patients_clean[['patient_id', 'given_name', 'surname']]
id_names.given_name = id_names.given_name.str.lower()
id_names.surname = id_names.surname.str.lower()
treatments_clean = pd.merge(treatments_clean, id_names, on=['given_name', 'surname'])
treatments_clean = treatments_clean.drop(['given_name', 'surname'], axis=1)

##### Test

In [ ]:
# Confirm the merge was executed correctly
treatments_clean

In [ ]:
# Patient ID should be the only duplicate column
all_columns = pd.Series(list(patients_clean) + list(treatments_clean))
all_columns[all_columns.duplicated()]